In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder.appName("Day 7 question 1").master("local[1]").enableHiveSupport().getOrCreate())

# Data Preparation for RDD

1.	Ranking of results – The below is a list of student information, including name, age, score.

In [3]:
students = [("Lilei", 18, 87),
            ("Hanmeimei", 16, 77),
            ("dachui", 16, 66),
            ("Jim", 18, 80),
            ("Ruhua", 20,50)
            ]

In [4]:
students_rdd = spark.sparkContext.parallelize(students)

### `(1)	Use RDD programming to find the top three students in the score.`

In [5]:
students_rdd.sortBy(lambda x: x[2], ascending=False) \
.top(3, lambda x: x[2])

[('Lilei', 18, 87), ('Jim', 18, 80), ('Hanmeimei', 16, 77)]

### `(3)	Use RDD programming to find out the students who rank first and last in the score.`

In [6]:
#rank first
students_rdd.sortBy(lambda x: x[2], ascending=False) \
.top(1, lambda x: x[2])

[('Lilei', 18, 87)]

In [7]:
#rank last
students_rdd.sortBy(lambda x: x[2], ascending=True) \
.first()

('Ruhua', 20, 50)

### `(5)	Use RDD programming to calculate the average score.`

In [8]:
total_score = students_rdd.map(lambda x: x[2]) \
.reduce(lambda x, y: x+y)
average_score = total_score/students_rdd.count()

In [9]:
average_score

72.0

# Data Preparation for spark-sql

1.	Ranking of results – The below is a list of student information, including name, age, score.

In [10]:
spark.sql("CREATE DATABASE IF NOT EXISTS day7")

DataFrame[]

In [11]:
spark.sql("USE day7")

DataFrame[]

In [12]:
spark.sql("""CREATE TABLE IF NOT EXISTS students(name string,age int,score int)""")

DataFrame[]

In [19]:
spark.sql("show tables").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|    day7| edu_admin|      false|
|    day7|edu_admin1|      false|
|    day7|  students|      false|
+--------+----------+-----------+



In [14]:
spark.sql("""insert into students values ('Lilei', 18, 87),
('Hanmeimei', 16, 77), 
('dachui', 16, 66),
('Jim', 18, 80),
('Ruhua', 20, 50)
""")

DataFrame[]

### `(2)	Use sparksql programming to find out the top 3 students in the score.`

In [15]:
spark.sql("select * from students").show()

+---------+---+-----+
|     name|age|score|
+---------+---+-----+
|    Lilei| 18|   87|
|Hanmeimei| 16|   77|
|   dachui| 16|   66|
|      Jim| 18|   80|
|    Ruhua| 20|   50|
+---------+---+-----+



In [16]:
spark.sql("""select *
from (
select *, dense_rank() over(order by score desc) as rank from students) t
where rank <= 3
""").show()

+---------+---+-----+----+
|     name|age|score|rank|
+---------+---+-----+----+
|    Lilei| 18|   87|   1|
|      Jim| 18|   80|   2|
|Hanmeimei| 16|   77|   3|
+---------+---+-----+----+



### `(4)	Use sparksql programming to find out the students who rank the first and penultimate in the score`

In [17]:
spark.sql("""select *
from (
select *, dense_rank() over (order by score desc) as rank from students) t
where rank == 1 or rank == 4
""").show()

+------+---+-----+----+
|  name|age|score|rank|
+------+---+-----+----+
| Lilei| 18|   87|   1|
|dachui| 16|   66|   4|
+------+---+-----+----+



### `(6)	Use sparksql programming to calculate the average score.`

In [18]:
spark.sql("""select avg(score) as Average_Score
from students
""").show()

+-------------+
|Average_Score|
+-------------+
|         72.0|
+-------------+

